In [ ]:
import vtk
import itk
from vtkmodules.vtkCommonCore import vtkCommand
filepath = "../human_body.nii"

In [2]:
# Read File
if ".nii" in filepath:
    itk_image = itk.imread(filepath)
    
    print(f"File: {('OK' if itk_image is not None else 'NOT OK')}")

File: OK


In [3]:
# Applying filter
itk_median_image = itk.median_image_filter(itk_image, radius=2)
vtk_image = itk.vtk_image_from_image(itk_median_image)
print(f"Filter: {('OK' if vtk_image is not None else 'NOT OK')}")

Filter: OK


In [4]:
# VTK Image Data to VTK Poly Data
marching_cubes = vtk.vtkMarchingCubes()
marching_cubes.SetInputData(vtk_image)
marching_cubes.SetValue(0, 100)
marching_cubes.Update()
print(f"VTK Poly Data: {('OK' if marching_cubes.GetOutput() is not None else 'NOT OK')}")

VTK Poly Data: OK


In [5]:
# Applying VTK contour filter
confilter = vtk.vtkPolyDataConnectivityFilter()
confilter.SetInputData(marching_cubes.GetOutput())
confilter.SetExtractionModeToLargestRegion()
confilter.Update()
print(f"VTK contour filter: {('OK' if confilter.GetOutput() is not None else 'NOT OK')}")

VTK contour filter: OK


In [6]:
# clipper
plane = vtk.vtkPlane()
clipper = vtk.vtkClipPolyData()
clipper.SetClipFunction(plane)
clipper.SetInputConnection(confilter.GetOutputPort())

In [7]:
# Visualization pipeline
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(clipper.GetOutputPort())
actor = vtk.vtkActor()
actor.SetMapper(mapper)

renderer = vtk.vtkRenderer()
ren_win = vtk.vtkRenderWindow()
ren_win.AddRenderer(renderer)

renderer.AddActor(actor)
colors = vtk.vtkNamedColors()
renderer.SetBackground(colors.GetColor3d('SlateGray'))

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(ren_win)

In [8]:
# Implicit plane widget
class IPWCallback:
    def __init__(self, plane):
        self.plane = plane

    def __call__(self, caller, ev):
        rep = caller.GetRepresentation()
        rep.GetPlane(self.plane)

rep = vtk.vtkImplicitPlaneRepresentation()
rep.SetPlaceFactor(1.25)
rep.PlaceWidget(actor.GetBounds())
rep.SetNormal(plane.GetNormal())


ipw_callback = IPWCallback(plane)
plane_widget = vtk.vtkImplicitPlaneWidget2()
plane_widget.SetInteractor(iren)
plane_widget.SetRepresentation(rep)
plane_widget.AddObserver(vtkCommand.InteractionEvent, ipw_callback)
plane_widget.On()

In [9]:
# Run
renderer.GetActiveCamera().Zoom(0.75)
renderer.ResetCamera()
iren.Initialize()
ren_win.Render()
iren.Start()